In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline


import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
## custom modules
import src.aformer_TF_gc_separated as aformer
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
import src.optimizers as optimizers
import src.schedulers as schedulers
import src.utils as utils

import training_utils_aformer_TF_genecentered_separated as training_utils


from scipy import stats

import os

os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'



In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-21')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 8
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2022-08-10 00:42:07.876527: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-08-10 00:42:07.876579: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-10 00:42:07.876603: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-9-20220803-154343): /proc/driver/nvidia/version does not exist
2022-08-10 00:42:07.990654: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler fla

INFO:tensorflow:Initializing the TPU system: node-21


INFO:tensorflow:Initializing the TPU system: node-21


In [ ]:
with strategy.scope():

    train_steps=100#3200#5165 #320
    warmup_steps=10
    val_steps_h=105#3200#757 ### 5562
    val_steps_m=14
    num_epochs=20
    lr_base=0.001
    warmup_lr=1.0e-06

    data_it_tr_list = []
    data_it_val_list = []

    ### create dataset iterators
    heads_dict = {}
    orgs = ["hg","mm"]
    for k, org in enumerate(orgs):
        heads_dict[org] = int(k)
    data_dict_tr,data_dict_val = training_utils.return_distributed_iterators(heads_dict,
                                                                             "gs://picard-testing-176520/16k_genecentered_blacklist0.50_atacnormalized/preprocessed",
                                                                              GLOBAL_BATCH_SIZE,
                                                                              16384,
                                                                              300,
                                                                              "logTPM",
                                                                              4,
                                                                              num_epochs,
                                                                              strategy,
                                                                              options,
                                                                              1637)

In [4]:
with strategy.scope():
    model = aformer.aformer(kernel_transformation="softmax_kernel_transformation",
                                dropout_rate=0.15,
                                input_length=16384,
                                num_heads=4,
                                numerical_stabilizer=0.0000001,
                                nb_random_features=256,
                                hidden_size=512,
                                d_model=512,
                                norm=True,
                                dim=64,
                                human=True,
                                max_seq_length = 128,
                                rel_pos_bins=128,
                                widening = 2, ## ratio between first and second dense layer units in transformer block
                                conv_filter_size_1_seq=25,
                                conv_filter_size_2_seq=5,
                                conv_filter_size_1_atac=50,
                                conv_filter_size_2_atac=5,
                            positional_dropout_rate=0.1,
                                transformer_depth=2,
                                momentum=0.90,
                                channels_list=[192,192,224,224,256,256], 
                                kernel_regularizer=0.000001,
                                TF_inputs_hg=1637,
                                TF_inputs_mm=1366,
                                use_mask_pos=False,
                                use_rot_emb=True,
                                heads_dict=heads_dict)
    

In [5]:
with strategy.scope():
    
    optimizer = tfa.optimizers.AdaBelief(learning_rate=0.001,
                                         weight_decay=5.0e-04,
                                         warmup_proportion=0.10,
                                         epsilon=1e-14,
                                         rectify=True,
                                         min_lr=0.000005,
                                         total_steps=10000)
    optimizer = tfa.optimizers.Lookahead(optimizer, sync_period=6, slow_step_size=0.5)
    """
    optimizer = tf.keras.optimizers.Adam(learning_rate =0.001)
    #optimizer = tfa.optimizers.Lookahead(optimizer, sync_period=6, slow_step_size=0.5)
    """

In [6]:
with strategy.scope():
    metric_dict = {}
    train_step, val_step, metric_dict = training_utils.return_train_val_functions(model,
                                                                                  orgs,
                                                                                 optimizer,
                                                                                 strategy,
                                                                                 metric_dict, 
                                                                                 train_steps,
                                                                                 val_steps_h,
                                                                                 val_steps_m,
                                                                                 GLOBAL_BATCH_SIZE,
                                                                                 0.2,
                                                                                 True,
                                                                                 fourier_loss_scale=0.5) # last is uncropped length


In [7]:
%matplotlib inline

In [ ]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 15):
        start = time.time()
        #if epoch_i > 2 : 
        train_step(data_dict_tr['hg'],
                   data_dict_tr['mm'])
                   #data_dict_tr['mm'],
                   #data_dict_tr['rm'])
        print('hg_train_loss: ' + str(metric_dict['hg_tr'].result().numpy()))
        val_step(data_dict_val['hg'], data_dict_val['mm'])

        val_losses.append(metric_dict['hg_val'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['hg_val'].result().numpy()))
        print('hg_val_pearson: ' + str(metric_dict['hg_corr_stats'].result()['pearsonR'].numpy()))
        print('hg_val_R2: ' + str(metric_dict['hg_corr_stats'].result()['R2'].numpy()))
        
        #print('hg_val_pearson_gene: ' + str(metric_dict['hg_corr_stats'].result()['feature_level_pearsonR'].numpy()))
        #print('hg_val_R2_gene: ' + str(metric_dict['hg_corr_stats'].result()['feature_level_R2'].numpy()))
        
        y_trues = metric_dict['hg_corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['hg_corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['hg_corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['hg_corr_stats'].result()['gene_map'].numpy()

        unique_preds = {}
        unique_trues = {}
        for k,x in enumerate(gene_map):
            unique_preds[(cell_types[k],x)] = y_preds[k]
            unique_trues[(cell_types[k],x)] = y_trues[k]
        
        unique_preds = dict(sorted(unique_preds.items()))
        unique_trues = dict(sorted(unique_trues.items()))
        
        
        print('overall gene correlation:', pearsonr(y_trues,
                                                    y_preds)[0])
        
        
        data = np.vstack([y_trues,y_preds])
        kernel = stats.gaussian_kde(data)(data)
        sns.scatterplot(
            x=y_trues,
            y=y_preds,
            c=kernel,
            cmap="viridis")
        plt.show()
        
        
        ### now compute correlations across cell types
        across_cells_preds = {}
        across_cells_trues = {}
        
        for k,v in unique_preds.items():
            cell_t,gene_name = k
            if cell_t not in across_cells_preds.keys():
                across_cells_preds[cell_t] = []
                across_cells_trues[cell_t] = []
            else:
                across_cells_preds[cell_t].append(v)
                across_cells_trues[cell_t].append(unique_trues[k])
        cell_specific_corrs = []
        for k,v in across_cells_preds.items():
            trues = []
            preds = []
            for idx,x in enumerate(v):
                preds.append(x)
                trues.append(across_cells_trues[k][idx])
            try: 
                cell_specific_corrs.append(pearsonr(trues, 
                                                    preds)[0])
            except np.linalg.LinAlgError:
                continue
            except ValueError:
                continue
                
        sns.histplot(x=np.asarray(cell_specific_corrs), bins=50)
        plt.show()
        print('median_cell_crossgenes:', np.nanmedian(cell_specific_corrs))
                
            
        ### now compute correlations across genes
        across_genes_preds = {}
        across_genes_trues = {}
        
        for k,v in unique_preds.items():
            cell_t,gene_name = k
            if gene_name not in across_genes_preds.keys():
                across_genes_preds[gene_name] = []
                across_genes_trues[gene_name] = []
            else:
                across_genes_preds[gene_name].append(v)
                across_genes_trues[gene_name].append(unique_trues[k])
        genes_specific_corrs = []
        genes_specific_vars = []
        for k,v in across_genes_preds.items():
            trues = []
            preds = []
            for idx, x in enumerate(v):
                #if len(x) > 0:
                preds.append(x)
                trues.append(across_genes_trues[k][idx])
            try: 
                genes_specific_corrs.append(pearsonr(trues, 
                                                     preds)[0])
                genes_specific_vars.append(np.var(trues))
            except np.linalg.LinAlgError:
                continue
            except ValueError:
                continue
                
        sns.histplot(x=np.asarray(genes_specific_corrs), bins=50)
        plt.show()
        print('median_gene_crossdataset:', np.nanmedian(genes_specific_corrs))
            
        sns.scatterplot(
            x=genes_specific_vars,
            y=genes_specific_corrs)
        plt.show()

        #print('mm_train_loss: ' + str(metric_dict['mm_tr'].result().numpy()))

        #print('mm_val_loss: ' + str(metric_dict['mm_val'].result().numpy()))
        #print('mm_val_pearson: ' + str(metric_dict['mm_corr_stats'].result()['pearsonR'].numpy()))
        #print('mm_val_R2: ' + str(metric_dict['mm_corr_stats'].result()['R2'].numpy()))
        
        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
    
        print('patience counter at: ' + str(patience_counter))

        for key, item in metric_dict.items():
            item.reset_state()

hg_train_loss: 0.5048983
hg_val_loss: 0.100430004
hg_val_pearson: 0.010387493
hg_val_R2: [-0.9853468]


In [ ]:
model.summary()

In [15]:
38,347,344

NameError: name 'y_true' is not defined

In [ ]:
def pearsons_batch(y_true, y_pred):
    '''
    Helper function to compute r2 for tensors of shape (batch, length)
    '''
    y_true.shape.assert_is_compatible_with(y_pred.shape)
    y_true = tf.cast(y_true, 'float32')
    y_pred = tf.cast(y_pred, 'float32')
    
    count = y_true.shape[1]
    product_sum = tf.reduce_sum(y_true * y_pred, axis=1,keepdims=True)

    sum_product = tf.reduce_sum(y_true, axis=1,keepdims=True) * tf.reduce_sum(y_pred, axis=1,keepdims=True)

    numerator = product_sum - (count * tf.reduce_mean(y_true,axis=1,keepdims=True) * tf.reduce_mean(y_pred,axis=1,keepdims=True))
    

    stdev_pred = tf.math.reduce_std(y_pred, axis=1,keepdims=True)
    stdev_true = tf.math.reduce_std(y_true, axis=1,keepdims=True)

    denominator = stdev_pred * stdev_true * tf.constant(count,dtype=tf.float32)
    
    pearsons = (numerator / denominator)[:,0]
    return pearsons

In [20]:

def r2_batch(y_true, y_pred):
    '''
    Helper function to compute r2 for tensors of shape (batch, length)
    to do: check descrepancy w/ tensorflow implementation
    '''
    y_true.shape.assert_is_compatible_with(y_pred.shape)
    y_true = tf.cast(y_true, 'float32')
    y_pred = tf.cast(y_pred, 'float32')
    residual = tf.reduce_sum(tf.square(y_true - y_pred),axis=1,keepdims=True)
    total = tf.reduce_sum(tf.square(y_true -  tf.reduce_mean(y_true)),axis=1,keepdims=True) + tf.constant(1.0e-06,dtype=tf.float32)
    r2 = tf.constant(1.0,dtype=tf.float32) - residual / total
    return r2#[:,0]

In [21]:
tf.reshape(r2_batch(y_true,y_pred), [-1])

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([-0.94959736, -1.0186756 , -1.139405  , -0.84549165], dtype=float32)>

In [23]:
r2_batch(y_true,y_pred)

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[-0.94959736],
       [-1.0186756 ],
       [-1.139405  ],
       [-0.84549165]], dtype=float32)>

In [27]:
test = tf.Variable([], shape=tf.TensorShape(None), validate_shape=False)

In [6]:
y_true = tf.random.normal([1,896], 1e-02, 1.0e-07, tf.float32, seed=1)
y_pred = tf.random.normal([1,896], 1e-02, 1.0e-07, tf.float32, seed=1)

In [4]:
out = data_dict_tr['hg']

In [23]:
test = next(out)
test['target']

PerReplica:{
  0: <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.]], dtype=float32)>,
  1: <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.4514349]], dtype=float32)>,
  2: <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.5739595]], dtype=float32)>,
  3: <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.0050988]], dtype=float32)>,
  4: <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.954692]], dtype=float32)>,
  5: <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.]], dtype=float32)>,
  6: <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.31018123]], dtype=float32)>,
  7: <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.635968]], dtype=float32)>
}

In [86]:
def pearsons_batch(y_true, y_pred):
    '''
    Helper function to compute r2 for tensors of shape (batch, length)
    '''
    y_true.shape.assert_is_compatible_with(y_pred.shape)
    y_true = tf.cast(y_true, 'float64')
    y_pred = tf.cast(y_pred, 'float64')
    
    true_sum = tf.reduce_sum(y_true,axis=1)
    pred_sum = tf.reduce_sum(y_pred,axis=1)
    
    count = tf.reduce_sum(tf.ones_like(y_true),axis=1)
    true_mean = true_sum / count
    #print(true_mean)
    true_sq_sum = tf.reduce_sum(tf.math.square(y_true),axis=1)
    pred_mean = pred_sum / count
    pred_sq_sum = tf.reduce_sum(tf.math.square(y_pred),axis=1)
    product_sum = tf.reduce_sum(y_true*y_pred, axis=1)
    
    covariance = (product_sum - true_mean * pred_sum
                      - pred_mean * true_sum
                      + count * true_mean * pred_mean)
    
    true_var = true_sq_sum - count * tf.math.square(true_mean)
    pred_var = pred_sq_sum - count * tf.math.square(pred_mean)
    tp_var = tf.math.sqrt(true_var) * tf.math.sqrt(pred_var)
    
    pearsons = covariance / tp_var
    #pearsons_not_nan = tf.dtypes.cast(tf.math.logical_not(tf.math.is_nan(pearsons)), dtype=tf.float32)
    return pearsons

def r2_batch(y_true, y_pred):
    '''
    Helper function to compute r2 for tensors of shape (batch, length)
    to do: check descrepancy w/ tensorflow implementation
    '''
    y_true.shape.assert_is_compatible_with(y_pred.shape)
    y_true = tf.cast(y_true, 'float64')
    y_pred = tf.cast(y_pred, 'float64')
    residual = tf.reduce_sum(tf.square(y_true - y_pred),axis=1,keepdims=True)
    total = tf.reduce_sum(tf.square(y_true -  tf.reduce_mean(y_true)),axis=1,keepdims=True)
    r2 = tf.constant(1.0,dtype=tf.float64) - residual / total
    return r2#[:,0]

In [87]:
y_true = tf.random.normal([4,896], 1e-07, 1.0e-08, tf.float64, seed=1)
y_pred = tf.random.normal([4,896], 1e-02, 1.0e-08, tf.float64, seed=2)

In [2]:
test = tf.constant('ATCG')
test_arr =  training_utils.one_hot(test)
test_arr

2022-07-04 17:52:58.965433: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-04 17:52:58.965472: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-04 17:52:58.965493: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist
2022-07-04 17:52:58.965841: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)>

In [4]:
test = tf.constant('cgat')
test_arr =  training_utils.rev_comp_one_hot(test)
test_arr

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]], dtype=float32)>

In [12]:
data_dict_val

{'hg': <tensorflow.python.distribute.input_lib.DistributedIterator at 0x7f851c504b90>,
 'mm': <tensorflow.python.distribute.input_lib.DistributedIterator at 0x7f84988d9090>}

In [13]:
dim=64
max_seq_len=1024


inv_freq = 1. / (10000 ** (tf.range(start=0, limit=dim, delta=2, dtype='float32') / dim))
position = tf.range(start=0, limit=max_seq_len, delta=1, dtype='float32')
sinusoid_inp = tf.einsum("i,j->ij", position, inv_freq)
emb = tf.concat((tf.math.sin(sinusoid_inp), tf.math.cos(sinusoid_inp)), axis=-1)
emb.shape

TensorShape([1024, 64])

In [24]:
def repeat(x, r):
    c = tf.ones_like(tf.shape(x), dtype=tf.int32)
    c1 = c[:-1]
    c2 = c[-1][None] * r
    c_ = tf.concat([c1, c2], axis=0)

    return tf.tile(x, c_)

dim=64
theta=10000
freqs = tf.convert_to_tensor(1. / (theta ** (np.arange(0, dim, 2)[:(dim // 2)] / dim)), dtype=tf.float32)

freqs.shape


TensorShape([32])

In [27]:
test = tf.ones((1024,64))

In [4]:
out = next(data_dict_tr['hg'])

In [5]:
out

{'inputs': PerReplica:{
   0: <tf.Tensor: shape=(4, 16384, 5), dtype=float32, numpy=
 array([[[0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0.]],
 
        [[1., 0., 0., 1., 0.],
         [1., 0., 0., 0., 1.],
         [1., 0., 0., 1., 0.],
         ...,
         [0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0.]],
 
        [[0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0.],
         ...,
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.]],
 
        [[0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         ...,
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 1., 0., 0., 0.]]], dtype=float32)>,
   1: <tf.Tensor: shape=(4, 16384, 5), dtype=float32, numpy=
 array([[[0., 0., 1., 0., 

In [15]:
test = tf.random.normal([1,32,128])
test

<tf.Tensor: shape=(1, 32, 128), dtype=float32, numpy=
array([[[ 0.4624173 ,  0.0709485 ,  0.9875072 , ...,  0.7712948 ,
         -0.44498843, -0.00588704],
        [ 0.19986393, -1.1558427 ,  1.3676628 , ...,  0.13729247,
          0.32130617,  0.04578542],
        [ 0.65867066,  1.6272559 ,  0.22596528, ...,  0.6174097 ,
          0.8941371 ,  0.6571273 ],
        ...,
        [-0.9744453 ,  2.1862698 , -1.9445254 , ..., -0.9426393 ,
         -0.29811418,  0.23624428],
        [ 0.20273659,  0.24291225, -0.40198046, ...,  0.42093408,
         -0.27420944, -0.51285267],
        [ 1.0759048 , -1.5710802 , -0.06107113, ..., -1.6967397 ,
          0.64171076,  0.82089496]]], dtype=float32)>

In [16]:
tf.tile(test, multiples=[32,1,1])

<tf.Tensor: shape=(32, 32, 128), dtype=float32, numpy=
array([[[ 0.4624173 ,  0.0709485 ,  0.9875072 , ...,  0.7712948 ,
         -0.44498843, -0.00588704],
        [ 0.19986393, -1.1558427 ,  1.3676628 , ...,  0.13729247,
          0.32130617,  0.04578542],
        [ 0.65867066,  1.6272559 ,  0.22596528, ...,  0.6174097 ,
          0.8941371 ,  0.6571273 ],
        ...,
        [-0.9744453 ,  2.1862698 , -1.9445254 , ..., -0.9426393 ,
         -0.29811418,  0.23624428],
        [ 0.20273659,  0.24291225, -0.40198046, ...,  0.42093408,
         -0.27420944, -0.51285267],
        [ 1.0759048 , -1.5710802 , -0.06107113, ..., -1.6967397 ,
          0.64171076,  0.82089496]],

       [[ 0.4624173 ,  0.0709485 ,  0.9875072 , ...,  0.7712948 ,
         -0.44498843, -0.00588704],
        [ 0.19986393, -1.1558427 ,  1.3676628 , ...,  0.13729247,
          0.32130617,  0.04578542],
        [ 0.65867066,  1.6272559 ,  0.22596528, ...,  0.6174097 ,
          0.8941371 ,  0.6571273 ],
        ...,

In [10]:
test = tf.cast(tf.constant([0,1,2,3,4,5,6,7,8,9]),dtype=tf.float32)

In [18]:
tf.nn.dropout(test, rate=0.2).numpy()*(1-0.2)

array([0., 0., 2., 0., 0., 5., 6., 7., 8., 9.], dtype=float32)